In [1]:
# pip install PuLP

In [2]:
from preamble import *

In [ ]:
import pickle

file_path = './data/pressure_flux_outflow_j.csv'

data = pd.read_csv(file_path, index_col='datetime')
data.index = pd.to_datetime(data.index)
hourly_data = data.resample('h').sum() / 60
hourly_data.reset_index(inplace=True)
hourly_data_one_day = hourly_data[:24]

In [4]:
data.head()

,pressure,flux,outflow
datetime,,,
2023-01-01 00:01:00,9.69,75.23,101.18
2023-01-01 00:02:00,9.69,74.65,98.53
2023-01-01 00:03:00,9.69,75.28,102.03
2023-01-01 00:04:00,9.69,74.67,100.53
2023-01-01 00:05:00,9.69,75.06,104.79


In [5]:
hourly_data.head()

,datetime,pressure,flux,outflow
0,2023-01-01 00:00:00,9.51,77.88,101.05
1,2023-01-01 01:00:00,9.57,107.09,119.92
2,2023-01-01 02:00:00,9.55,143.67,124.54
3,2023-01-01 03:00:00,9.57,144.95,80.16
4,2023-01-01 04:00:00,9.31,159.06,104.88


In [6]:
hourly_data_one_day.head()

,datetime,pressure,flux,outflow
0,2023-01-01 00:00:00,9.51,77.88,101.05
1,2023-01-01 01:00:00,9.57,107.09,119.92
2,2023-01-01 02:00:00,9.55,143.67,124.54
3,2023-01-01 03:00:00,9.57,144.95,80.16
4,2023-01-01 04:00:00,9.31,159.06,104.88


In [7]:
from datetime import datetime, time

electricity_rates = {
  'summer': {
    'off_peak': 64.37,  # 경부하 요금 (원/kWh)
    'mid_peak': 92.46,  # 중간부하 요금 (원/kWh)
    'on_peak': 123.88   # 최대부하 요금 (원/kWh)
  },
  'spring_fall': {
    'off_peak': 64.37,  # 경부하 요금 (원/kWh)
    'mid_peak': 69.50,  # 중간부하 요금 (원/kWh)
    'on_peak': 86.88    # 최대부하 요금 (원/kWh)
  },
  'winter': {
    'off_peak': 71.88,  # 경부하 요금 (원/kWh)
    'mid_peak': 90.80,  # 중간부하 요금 (원/kWh)
    'on_peak': 116.47   # 최대부하 요금 (원/kWh)
  }
}

# 시간대 구분
time_periods = {
  'summer': {
    'off_peak': [(time(23, 0), time(9, 0))],
    'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
    'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
  },
  'spring_fall': {
    'off_peak': [(time(23, 0), time(9, 0))],
    'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
    'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
  },
  'winter': {
    'off_peak': [(time(23, 0), time(9, 0))],
    'mid_peak': [(time(9, 0), time(10, 0)), (time(12, 0), time(17, 0)), (time(20, 0), time(22, 0))],
    'on_peak': [(time(10, 0), time(12, 0)), (time(17, 0), time(20, 0)), (time(22, 0), time(23, 0))]
  }
}

In [8]:
def get_season_and_period(dt):
  month = dt.month
  current_time = dt.time()
  
  if month in [7, 8]:
    season = 'summer'
  elif month in [11, 12, 1, 2]:
    season = 'winter'
  else:
    season = 'spring_fall'
  
  for period, times in time_periods[season].items():
    for start, end in times:
      # 자정을 넘어가는 경우 처리
      if start > end:  # 예: 23:00 ~ 09:00
        if start <= current_time or current_time < end:
          return season, period
      else:
        if start <= current_time < end:
          return season, period
  return season, 'off_peak'

In [9]:
def get_hourly_rates(start_datetime):
  """주어진 시작 시간부터 24시간 동안의 시간별 전기 요금을 반환합니다."""
  hourly_rates = []
  for hour in range(24):
    current_dt = start_datetime.replace(hour=(start_datetime.hour + hour) % 24)
    season, period = get_season_and_period(current_dt)
    hourly_rates.append(electricity_rates[season][period])
  return hourly_rates

In [18]:
import pulp as pl

def optimize_inflow(water_level, capacity, outflow, start_datetime, model_path, max_flow=250):
    with open(model_path, 'rb') as f:
        model, poly = pickle.load(f)

    min_safe_level = capacity * 0.3
    max_safe_level = capacity * 0.95
    hourly_rates = get_hourly_rates(start_datetime)

    prob = pl.LpProblem("Reservoir_Inflow_Optimization", pl.LpMinimize)
    inflow = [pl.LpVariable(f"inflow_{h}", lowBound=0, upBound=max_flow) for h in range(24)]

    # 목적 함수: 전기 요금 최소화
    prob += pl.lpSum([inflow[h] * hourly_rates[h] for h in range(24)])

    # 제약 조건 1: 24시간 동안 저수량이 안전 범위를 벗어나지 않도록 강제
    current_level = water_level
    for h in range(24):
        current_level = current_level + inflow[h] - outflow[h]
        prob += current_level >= min_safe_level  # 안전 하한 강제
        prob += current_level <= max_safe_level  # 안전 상한 강제

    # 제약 조건 2: 오전 9시에 max_safe_level을 초과하지 않도록 강제
    target_hour = 9
    hours_to_target = (target_hour - start_datetime.hour) % 24
    level_at_9am = water_level + pl.lpSum([inflow[h] - outflow[h] for h in range(hours_to_target)])

    prob += level_at_9am <= max_safe_level  # 초과 금지

    # 부족한 부분을 최소화하기 위한 변수 추가
    deviation_below_9am = pl.LpVariable("deviation_below_9am", lowBound=0)

    # 목표보다 부족한 경우 deviation_below_9am 값에 저장
    prob += level_at_9am + deviation_below_9am >= max_safe_level  # 부족한 부분을 최소화

    # 목적 함수에 페널티 추가 (부족한 부분 최소화 유도)
    penalty_weight = 1000  # 전기 요금보다 큰 가중치 부여
    prob += penalty_weight * deviation_below_9am

    prob.solve(pl.PULP_CBC_CMD(msg=False))
    optimal_inflow = [inflow[h].value() for h in range(24)]

    return optimal_inflow

In [19]:
def calculate_total_cost(inflow, start_datetime, model_path):
  """다항 회귀 모델을 사용하여 실제 전기 요금을 계산합니다."""
  with open(model_path, 'rb') as f:
    model, poly = pickle.load(f)
  
  total_cost = 0
  for h in range(24):
    current_dt = start_datetime.replace(hour=(start_datetime.hour + h) % 24)
    season, period = get_season_and_period(current_dt)
    
    # 다항 특성 변환
    flux_poly = poly.transform(np.array([[inflow[h]]]))
    
    # 전력 사용량 예측
    power_kW = model.predict(flux_poly)[0]
    
    # 해당 시간대의 요금
    cost_per_kWh = electricity_rates[season][period]
    
    # 시간당 비용 계산
    hourly_cost = power_kW * cost_per_kWh
    total_cost += hourly_cost
  
  return total_cost

In [20]:
def simulate_water_levels(water_level, capacity, inflow, outflow):
  """유입량과 유출량에 따른 저수량 변화를 시뮬레이션합니다."""
  levels = [water_level]
  min_safe_level = capacity * 0.3
  max_safe_level = capacity * 0.95
  
  for h in range(24):
    new_level = levels[-1] + inflow[h] - outflow[h]
    levels.append(new_level)
    
    if new_level < min_safe_level - 0.1 or new_level > max_safe_level + 0.1:
      print(f"경고: 시간 {h+1}에 저수량({new_level})이 안전 범위를 벗어났습니다.")
  
  return levels[:-2]

In [21]:
water_level = 1419  # 현재 저수량
capacity = 2000    # 최대 저수량
model_path = "./model/best_polynomial_regression_model.pkl"  # 다항 회귀 모델 경로
start_datetime = datetime(2023, 1, 1, 0, 0)  # 시작 날짜 및 시간

data_opti = hourly_data_one_day.copy()
outflow = data_opti['outflow']

inflow = data_opti['flux']

optimal_inflow = optimize_inflow(water_level, capacity, outflow, start_datetime, model_path, max_flow=250)

total_actual_cost = calculate_total_cost(inflow, start_datetime, model_path)
total_opti_cost = calculate_total_cost(optimal_inflow, start_datetime, model_path)

final_levels = simulate_water_levels(water_level, capacity, optimal_inflow, outflow)

# 결과 출력
print("최적 유입량:", [round(flow, 2) for flow in optimal_inflow])
print("최종 저수량:", [round(level, 2) for level in final_levels])


cost_format = format(int(total_actual_cost), ',')
cost_format_opti = format(int(total_opti_cost), ',')
saved_cost = total_actual_cost - total_opti_cost
saved_cost_format = format(int(saved_cost), ',')
print(f'Total daily electricity cost: {cost_format} KRW')
print(f'Total daily electricity cost used optimization : {cost_format_opti} KRW')
print(f'Save money on electricity bills : {saved_cost_format} KRW')
print(f'electricity bill savings rate : {(1 - (int(total_opti_cost) / int(total_actual_cost))) * 100:.2f} % 감소')

최적 유입량: [250.0, 250.0, 250.0, 156.68, 104.88, 117.73, 91.1, 0.0, 250.0, 0.0, 0.0, 0.0, 250.0, 250.0, 0.0, 250.0, 0.0, 175.1, 250.0, 250.0, 0.0, 0.0, 0.0, 0.0]
최종 저수량: [1419, 1567.95, 1698.03, 1823.49, 1900.0, 1900.0, 1900.0, 1884.34, 1770.9, 1900.0, 1744.14, 1559.7, 1336.34, 1302.1, 1308.13, 1078.95, 1176.29, 1042.16, 1033.99, 1150.12, 1273.17, 1074.91, 868.5]
Total daily electricity cost: 352,645 KRW
Total daily electricity cost used optimization : 304,220 KRW
Save money on electricity bills : 48,425 KRW
electricity bill savings rate : 13.73 % 감소


c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.6.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PolynomialFeatures from version 1.6.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective